In [1]:
!pip install datasets
!pip install datasets
!pip install evaluate rouge_score
!pip install accelerate

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=cf34096dd737bc2d88e7063a1a09bf942ec16511650de389d121c80b021f6597
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import math
import re
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import numpy as np
import pandas as pd
import accelerate

2024-06-02 20:46:46.617266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 20:46:46.617360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 20:46:46.852308: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
dataset = load_dataset("Valleyy/final_nlp_data")

Generating train split:   0%|          | 0/8363 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/2091 [00:00<?, ? examples/s]

In [4]:
train_data = dataset["train"]['Segmented_document']

valid_data = dataset["valid"]['Segmented_document']

In [5]:
import pandas as pd

# Assuming you have loaded the dataset as "dataset"
train_df = dataset["train"].to_pandas()
valid_df = dataset["valid"].to_pandas()


# **MMR with TF-IDF**

In [6]:
import math
import re
import pandas as pd

# Function to process the file text into sentences
def processFile(file_text):
    sentences = re.split(r'\.\s*', file_text)
    sentences = [sentence for sentence in sentences if sentence]
    return sentences

# Function to calculate term frequency
def TFs(sentences):
    word_freq = {}
    for sentence in sentences:
        tokens = sentence.split()
        for token in tokens:
            if token not in word_freq:
                word_freq[token] = 1
            else:
                word_freq[token] += 1
    return word_freq

# Function to calculate inverse document frequency
def IDFs(sentences):
    doc_freq = {}
    for sentence in sentences:
        tokens = sentence.split()
        for token in set(tokens):
            if token not in doc_freq:
                doc_freq[token] = 1
            else:
                doc_freq[token] += 1

    N = len(sentences)
    IDF = {token: math.log(N / freq) for token, freq in doc_freq.items()}
    return IDF

# Function to calculate TF-IDF
def TF_IDF(sentences):
    TFs_w = TFs(sentences)
    IDFs_w = IDFs(sentences)
    TF_IDF_w = {word: freq * IDFs_w[word] for word, freq in TFs_w.items()}
    return TF_IDF_w

# Function to calculate sentence similarity using TF-IDF
def sentenceSim(sentence1, sentence2, IDF_w):
    tokens1 = sentence1.split()
    tokens2 = sentence2.split()
    vector1 = [IDF_w.get(token, 0) * tokens1.count(token) for token in tokens1]
    vector2 = [IDF_w.get(token, 0) * tokens2.count(token) for token in tokens2]
    
    numerator = sum(x * y for x, y in zip(vector1, vector2))
    denom1 = math.sqrt(sum(x ** 2 for x in vector1))
    denom2 = math.sqrt(sum(y ** 2 for y in vector2))

    if denom1 == 0 or denom2 == 0:
        return 0.0

    return numerator / (denom1 * denom2)

# Function to build a query from the top TF-IDF words
def buildQuery(sentences, TF_IDF_w, n):
    sorted_words = sorted(TF_IDF_w, key=TF_IDF_w.get, reverse=True)
    query_words = sorted_words[:n]
    return " ".join(query_words)

# Function to find the best sentence matching the query
def bestSentence(sentences, query, IDF):
    best_sentence = None
    max_sim = -1
    for sentence in sentences:
        sim = sentenceSim(sentence, query, IDF)
        if sim > max_sim:
            max_sim = sim
            best_sentence = sentence
    return best_sentence

# Function to calculate MMR score
def MMRScore(Si, query, Sj_list, lambta, IDF):
    Sim1 = sentenceSim(Si, query, IDF)
    Sim2_max = max([sentenceSim(Si, Sj, IDF) for Sj in Sj_list])
    return lambta * Sim1 - (1 - lambta) * Sim2_max

# Function to generate a summary using MMR
def makeSummary(sentences, best_sentence, query, lambta, IDF, max_words=256):
    def word_count(text):
        return len(text.split())

    if word_count(" ".join(sentences)) <= max_words:
        return ". ".join(sentences)

    indexed_sentences = list(enumerate(sentences))
    best_sentence_index = sentences.index(best_sentence)
    summary_indices = [best_sentence_index]
    indexed_sentences.remove((best_sentence_index, best_sentence))

    current_summary = sentences[best_sentence_index]

    while word_count(current_summary) < max_words:
        MMR_scores = {index: MMRScore(sentence, query, best_sentence, lambta, IDF) for index, sentence in indexed_sentences}
        
        if not MMR_scores:
            break

        selected_index, _ = max(MMR_scores.items(), key=lambda x: x[1])
        selected_sentence = [s for i, s in indexed_sentences if i == selected_index][0]

        if word_count(current_summary + " " + selected_sentence) > max_words:
            break

        summary_indices.append(selected_index)
        indexed_sentences = [(i, s) for i, s in indexed_sentences if i != selected_index]

        best_sentence = selected_sentence
        current_summary += " " + selected_sentence

    summary_indices.sort()
    summary = [sentences[i] for i in summary_indices]

    return ". ".join(summary)

# Function to add the importance_sentences column
def add_importance_sentences(df):
    importance_sentences = []

    for file_text in df['Segmented_document']:
        processed_sentences = processFile(file_text)
        TF_IDF_w = TF_IDF(processed_sentences)
        query = buildQuery(processed_sentences, TF_IDF_w, n=20)
        IDF = IDFs(processed_sentences)
        best_sent = bestSentence(processed_sentences, query, IDF)
        summary = makeSummary(processed_sentences, best_sent, query, lambta=0.7, IDF=IDF)
        importance_sentences.append(summary)

    df['importance_sentences'] = importance_sentences
    return df

# Main function to process the dataset and save to CSV
def main():
    # Load your dataset
    # Assuming you have loaded the dataset as "dataset"
    # Convert dataset splits to pandas DataFrames
    train_df = dataset["train"].to_pandas()
    valid_df = dataset["valid"].to_pandas()

    # Add the importance_sentences column
    train_df = add_importance_sentences(train_df)
    valid_df = add_importance_sentences(valid_df)

    # Save the updated DataFrames to CSV
    train_df.to_csv('train_with_importance_sentences.csv', index=False)
    valid_df.to_csv('valid_with_importance_sentences.csv', index=False)

if __name__ == "__main__":
    main()


In [7]:
import pandas as pd

# Define the file path
file_path = '/kaggle/working/train_with_importance_sentences.csv'

# Read the CSV file into a DataFrame
train_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(train_df.head())


                                            Document  \
0  Thiếu tướng Trần Kỳ Rơi , giám đốc Công an tỉn...   
1  Hiện trường vụ tai nạn .Khoảng 21h ngày 14/11 ...   
2  "Không biết công văn của Sở Y tế trả lời cơ qu...   
3  Liên quan vụ việc đối tượng Bùi Xuân Hồng ( SN...   
4  Chiều 25.2, Tổ công tác do Thượng uý Hoàng Tùn...   

                                             Summary  \
0  Liên quan đến vụ 500 giáo viên dôi dư tại Đắk ...   
1  Tài xế chạy với tốc độ cao , không làm chủ tay...   
2  Bị cáo Hoàng Công Lương cho rằng, hôm xảy ra s...   
3  Tại cơ quan Công an , nghi phạm Bùi Xuân Hồng ...   
4  Sau khi thông báo về lỗi vi phạm, tài xế ở Hà ...   

                                  Segmented_document  \
0  Thiếu_tướng Trần_Kỳ_Rơi , giám_đốc Công_an tỉn...   
1  Hiện_trường vụ tai_nạn . Khoảng 21h ngày 14/11...   
2  " Không biết công_văn của Sở Y_tế trả_lời cơ_q...   
3  Liên_quan vụ_việc đối_tượng Bùi_Xuân_Hồng ( SN...   
4  Chiều 25.2 , Tổ công_tác do Thượng_uý Hoàng